# Mehanizmi pozornosti in transformatorji

Ena glavnih pomanjkljivosti rekurzivnih omrežij je, da imajo vse besede v zaporedju enak vpliv na rezultat. To povzroča suboptimalno delovanje pri standardnih modelih LSTM kodirnik-dekodirnik za naloge zaporedja v zaporedje, kot so prepoznavanje imenovanih entitet in strojno prevajanje. V resnici imajo določene besede v vhodnem zaporedju pogosto večji vpliv na izhodne rezultate kot druge.

Razmislimo o modelu zaporedja v zaporedje, kot je strojno prevajanje. Ta model je implementiran z dvema rekurzivnima omrežjema, kjer eno omrežje (**kodirnik**) stisne vhodno zaporedje v skrito stanje, drugo omrežje (**dekodirnik**) pa razširi to skrito stanje v preveden rezultat. Težava pri tem pristopu je, da ima končno stanje omrežja težave pri pomnjenju začetka stavka, kar povzroča slabšo kakovost modela pri dolgih stavkih.

**Mehanizmi pozornosti** omogočajo tehtanje kontekstualnega vpliva vsakega vhodnega vektorja na vsako izhodno napoved RNN. To se implementira z ustvarjanjem bližnjic med vmesnimi stanji vhodnega RNN in izhodnega RNN. Na ta način, ko generiramo izhodni simbol $y_t$, upoštevamo vsa skrita stanja vhodov $h_i$ z različnimi utežnimi koeficienti $\alpha_{t,i}$.

![Slika prikazuje model kodirnik/dekodirnik z dodatno plastjo pozornosti](../../../../../translated_images/sl/encoder-decoder-attention.7a726296894fb567.webp)
*Model kodirnik-dekodirnik z mehanizmom dodatne pozornosti v [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), citirano iz [tega bloga](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Matrika pozornosti $\{\alpha_{i,j}\}$ predstavlja stopnjo, do katere določene vhodne besede vplivajo na generacijo določene besede v izhodnem zaporedju. Spodaj je primer takšne matrike:

![Slika prikazuje vzorčno poravnavo, ki jo je našel RNNsearch-50, vzeto iz Bahdanau - arviz.org](../../../../../translated_images/sl/bahdanau-fig3.09ba2d37f202a6af.webp)

*Slika vzeta iz [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Mehanizmi pozornosti so odgovorni za velik del trenutnega ali skoraj trenutnega stanja umetnosti na področju obdelave naravnega jezika. Dodajanje pozornosti pa močno poveča število parametrov modela, kar je povzročilo težave pri skaliranju RNN-jev. Ključna omejitev skaliranja RNN-jev je, da rekurzivna narava modelov otežuje združevanje in paralelizacijo učenja. Pri RNN mora biti vsak element zaporedja obdelan v zaporednem vrstnem redu, kar pomeni, da ga ni mogoče enostavno paralelizirati.

Uporaba mehanizmov pozornosti v kombinaciji s to omejitvijo je privedla do nastanka transformatorjev, ki so trenutno stanje umetnosti in jih danes poznamo ter uporabljamo, od BERT do OpenGPT3.

## Transformatorji

Namesto da bi prenašali kontekst vsake prejšnje napovedi v naslednji korak ocenjevanja, **transformatorji** uporabljajo **pozicijske kodiranja** in **pozornost**, da zajamejo kontekst danega vhoda znotraj določenega okna besedila. Spodnja slika prikazuje, kako pozicijska kodiranja s pozornostjo zajamejo kontekst znotraj določenega okna.

![Animiran GIF prikazuje, kako se ocene izvajajo v transformatorjih.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Ker je vsaka vhodna pozicija neodvisno preslikana v vsako izhodno pozicijo, transformatorji omogočajo boljšo paralelizacijo kot RNN-ji, kar omogoča veliko večje in bolj izrazne jezikovne modele. Vsaka glava pozornosti se lahko uporablja za učenje različnih odnosov med besedami, kar izboljša naloge obdelave naravnega jezika.

## Gradnja preprostega modela transformatorja

Keras ne vsebuje vgrajene plasti transformatorja, vendar jo lahko zgradimo sami. Kot prej se bomo osredotočili na klasifikacijo besedila iz nabora podatkov AG News, vendar je vredno omeniti, da transformatorji kažejo najboljše rezultate pri težjih nalogah obdelave naravnega jezika.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Nove plasti v Kerasu morajo podrazrediti razred `Layer` in implementirati metodo `call`. Začnimo s plastjo **Positional Embedding**. Uporabili bomo [nekaj kode iz uradne dokumentacije Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Predpostavljali bomo, da zapolnimo vse vhodne zaporedja na dolžino `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Ta plast je sestavljena iz dveh `Embedding` plasti: za vdelavo tokenov (na način, ki smo ga že obravnavali) in položajev tokenov. Položaji tokenov so ustvarjeni kot zaporedje naravnih števil od 0 do `maxlen` z uporabo `tf.range`, nato pa so posredovani skozi vdelano plast. Dva nastala vdelana vektorja se nato seštejeta, kar ustvari položajsko vdelano predstavitev vhodnih podatkov oblike `maxlen`$\times$`embed_dim`.

Zdaj pa implementirajmo transformacijski blok. Ta bo sprejel izhod prej definirane vdelane plasti:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Zdaj smo pripravljeni definirati celoten model transformatorja:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT Transformer modeli

**BERT** (Bidirectional Encoder Representations from Transformers) je zelo velik večplastni transformatorni model z 12 plastmi za *BERT-base* in 24 za *BERT-large*. Model je najprej predhodno usposobljen na velikem korpusu besedilnih podatkov (WikiPedia + knjige) z uporabo nenadzorovanega učenja (napovedovanje zakritih besed v stavku). Med predhodnim usposabljanjem model pridobi pomembno raven razumevanja jezika, ki jo je nato mogoče uporabiti z drugimi nabori podatkov prek finega prilagajanja. Ta proces se imenuje **prenosno učenje**.

![slika s http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/sl/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Obstaja veliko različic transformatornih arhitektur, vključno z BERT, DistilBERT, BigBird, OpenGPT3 in drugimi, ki jih je mogoče fino prilagoditi.

Poglejmo, kako lahko uporabimo predhodno usposobljen model BERT za reševanje našega tradicionalnega problema klasifikacije zaporedij. Idejo in nekaj kode bomo vzeli iz [uradne dokumentacije](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Za nalaganje predhodno usposobljenih modelov bomo uporabili **Tensorflow hub**. Najprej naložimo BERT-specifični vektorizator:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Pomembno je, da uporabite isti vektorizator, kot je bil uporabljen pri treniranju originalnega omrežja. Poleg tega BERT vektorizator vrne tri komponente:
* `input_word_ids`, ki je zaporedje številk tokenov za vhodni stavek
* `input_mask`, ki prikazuje, kateri del zaporedja vsebuje dejanski vhod in kateri del je polnilo. To je podobno maski, ki jo ustvari plast `Masking`
* `input_type_ids` se uporablja za naloge jezikovnega modeliranja in omogoča določitev dveh vhodnih stavkov v enem zaporedju.

Nato lahko ustvarimo BERT ekstraktor značilnosti:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Torej, BERT plast vrne več uporabnih rezultatov:
* `pooled_output` je rezultat povprečenja vseh tokenov v zaporedju. Lahko ga razumete kot inteligentno semantično vdelavo celotnega omrežja. To je enakovredno izhodu sloja `GlobalAveragePooling1D` v našem prejšnjem modelu.
* `sequence_output` je izhod zadnje transformer plasti (ustreza izhodu `TransformerBlock` v našem zgornjem modelu).
* `encoder_outputs` so izhodi vseh transformer plasti. Ker smo naložili 4-slojni BERT model (kot verjetno lahko sklepate iz imena, ki vsebuje `4_H`), ima 4 tenzorje. Zadnji je enak kot `sequence_output`.

Zdaj bomo definirali končni model za klasifikacijo. Uporabili bomo *funkcionalno definicijo modela*, kjer definiramo vhod modela in nato podamo niz izrazov za izračun njegovega izhoda. Prav tako bomo uteži BERT modela nastavili kot netrenirljive in trenirali samo končni klasifikator:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Kljub temu, da je malo parametrov za učenje, je proces precej počasen, saj je BERT-ov ekstraktor značilnosti računsko zahteven. Zdi se, da nismo uspeli doseči zadovoljive natančnosti, bodisi zaradi pomanjkanja učenja ali pomanjkanja parametrov modela.

Poskusimo odmrzniti uteži BERT-a in ga tudi trenirati. To zahteva zelo majhno hitrost učenja ter bolj previdno strategijo učenja z **ogrevanjem** in uporabo optimizatorja **AdamW**. Za ustvarjanje optimizatorja bomo uporabili paket `tf-models-official`:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Kot lahko vidiš, gre učenje precej počasi – vendar bi morda želel eksperimentirati in model trenirati za nekaj epochov (5–10) ter preveriti, ali lahko dosežeš boljši rezultat v primerjavi s pristopi, ki smo jih uporabili prej.

## Knjižnica Huggingface Transformers

Drug zelo pogost (in nekoliko enostavnejši) način uporabe Transformer modelov je [HuggingFace paket](https://github.com/huggingface/), ki ponuja preproste gradnike za različne naloge obdelave naravnega jezika (NLP). Na voljo je tako za Tensorflow kot za PyTorch, še en zelo priljubljen okvir za nevronske mreže.

> **Opomba**: Če te ne zanima, kako deluje knjižnica Transformers, lahko preskočiš na konec tega zvezka, saj ne boš videl ničesar bistveno drugačnega od tega, kar smo naredili zgoraj. Ponovili bomo iste korake treniranja BERT modela z uporabo druge knjižnice in bistveno večjega modela. Zato proces vključuje precej dolgotrajno učenje, tako da morda želiš le preleteti kodo.

Poglejmo, kako lahko naš problem rešimo z uporabo [Huggingface Transformers](http://huggingface.co).


Najprej moramo izbrati model, ki ga bomo uporabljali. Poleg nekaterih vgrajenih modelov Huggingface vsebuje [spletni repozitorij modelov](https://huggingface.co/models), kjer lahko najdete veliko več predhodno naučenih modelov, ki jih je prispevala skupnost. Vse te modele je mogoče naložiti in uporabljati zgolj z navedbo imena modela. Vsi potrebni binarni datoteki za model se bodo samodejno prenesli.

V določenih primerih boste morali naložiti svoje modele, v tem primeru lahko določite imenik, ki vsebuje vse ustrezne datoteke, vključno s parametri za tokenizer, datoteko `config.json` s parametri modela, binarne uteži itd.

Iz imena modela lahko ustvarimo tako model kot tokenizer. Začnimo s tokenizerjem:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` objekt vsebuje funkcijo `encode`, ki jo je mogoče neposredno uporabiti za kodiranje besedila:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Prav tako lahko uporabimo tokenizer za kodiranje zaporedja na način, ki je primeren za posredovanje modelu, tj. vključno s polji `token_ids`, `input_mask` itd. Prav tako lahko določimo, da želimo Tensorflow tenzorje, tako da podamo argument `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

V našem primeru bomo uporabili vnaprej naučen BERT model z imenom `bert-base-uncased`. *Uncased* pomeni, da je model neobčutljiv na velike in male črke.

Pri treniranju modela moramo kot vhod zagotoviti tokenizirano zaporedje, zato bomo zasnovali podatkovno procesno cevovod. Ker je `tokenizer.encode` Python funkcija, bomo uporabili enak pristop kot v prejšnji enoti, kjer jo pokličemo z uporabo `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Zdaj lahko naložimo dejanski model z uporabo paketa `BertForSequenceClassification`. To zagotavlja, da ima naš model že zahtevano arhitekturo za klasifikacijo, vključno s končnim klasifikatorjem. Videli boste opozorilno sporočilo, ki navaja, da uteži končnega klasifikatorja niso inicializirane in da model potrebuje predhodno učenje - to je povsem v redu, saj je točno to tisto, kar bomo storili!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Kot lahko vidite iz `summary()`, model vsebuje skoraj 110 milijonov parametrov! Predvidoma, če želimo preprosto nalogo klasifikacije na razmeroma majhnem naboru podatkov, ne želimo trenirati osnovnega sloja BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Zdaj smo pripravljeni na začetek usposabljanja!

> **Opomba**: Usposabljanje celotnega modela BERT lahko vzame veliko časa! Zato ga bomo usposabljali le za prvih 32 serij. To je zgolj za prikaz, kako je usposabljanje modela nastavljeno. Če vas zanima polno usposabljanje - preprosto odstranite parametra `steps_per_epoch` in `validation_steps` ter se pripravite na čakanje!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Če povečate število iteracij in počakate dovolj dolgo ter trenirate skozi več epochov, lahko pričakujete, da bo klasifikacija z BERT dosegla najboljšo natančnost! To je zato, ker BERT že zelo dobro razume strukturo jezika, zato je potrebno le prilagoditi končni klasifikator. Vendar pa, ker je BERT velik model, celoten proces učenja traja dolgo in zahteva veliko računske moči! (GPU, in po možnosti več kot enega).

> **Opomba:** V našem primeru uporabljamo enega najmanjših vnaprej naučenih modelov BERT. Obstajajo večji modeli, ki bodo verjetno dali boljše rezultate.


## Ključne točke

V tej enoti smo si ogledali zelo nedavne arhitekture modelov, ki temeljijo na **transformerjih**. Uporabili smo jih za našo nalogo klasifikacije besedila, vendar lahko modele BERT podobno uporabimo tudi za ekstrakcijo entitet, odgovarjanje na vprašanja in druge naloge obdelave naravnega jezika (NLP).

Transformer modeli predstavljajo trenutno najsodobnejši pristop v NLP-ju in v večini primerov bi morali biti prva rešitev, s katero začnete eksperimentirati pri implementaciji prilagojenih NLP rešitev. Kljub temu je razumevanje osnovnih načel rekurentnih nevronskih mrež, o katerih smo govorili v tem modulu, izjemno pomembno, če želite graditi napredne nevronske modele.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za prevajanje z umetno inteligenco [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem maternem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo profesionalni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazume ali napačne razlage, ki bi nastale zaradi uporabe tega prevoda.
